In [1]:
import typing as t
from pathlib import Path

import nltk
import pandas as pd
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm

In [2]:
tqdm.pandas()

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
DATASET_DIR = Path('/root/data')
FP_DATASET_DIR = DATASET_DIR / 'feedback-prize-effectiveness'

In [5]:
train_df = pd.read_csv(FP_DATASET_DIR / 'train_with_pos.csv')

In [6]:
class _EssayTextGetter:
    _sws = stopwords.words('english') + ['n\'t',  '\'s', '\'ve']

    def __init__(self, preprocess: bool = False):
        self._preprocess = preprocess

    def __call__(self, essay_id: str) -> str:
        with open(FP_DATASET_DIR / f'train/{essay_id}.txt') as f:
            essay_text = f.read().strip()
        if self._preprocess:
            word_tokens = word_tokenize(essay_text)
            essay_text = ' '.join([w for w in word_tokens if not w.lower() in self._sws])
        return essay_text


train_df['essay_text'] = train_df['essay_id'].progress_apply(_EssayTextGetter())
train_df['essay_text_pp'] = train_df['essay_id'].progress_apply(_EssayTextGetter())

  0%|          | 0/36765 [00:00<?, ?it/s]

  0%|          | 0/36765 [00:00<?, ?it/s]

In [7]:
def get_topics(text_list: t.List[str]):
    topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=5, verbose=True)
    topics, probs = topic_model.fit_transform(text_list)
    return topics, probs

In [8]:
topics, probs = get_topics(train_df['essay_text'].to_list())

Batches:   0%|          | 0/1149 [00:00<?, ?it/s]

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Jul  7 01:29:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/